In [2]:
import pydeck as pdk
import pandas as pd
import numpy as np


# Generate random data for raindrops
def generate_raindrops(num_drops, lat_range, lon_range, height_range):
    lats = np.random.uniform(lat_range[0], lat_range[1], num_drops)
    lons = np.random.uniform(lon_range[0], lon_range[1], num_drops)
    heights = np.random.uniform(height_range[0], height_range[1], num_drops)
    sizes = np.random.uniform(1, 5, num_drops)  # Varying sizes of raindrops

    data = pd.DataFrame({"lat": lats, "lon": lons, "height": heights, "size": sizes})
    return data


# Parameters for raindrops
num_drops = 1000
lat_range = [37.70, 37.80]  # Latitude range for raindrops
lon_range = [-122.50, -122.35]  # Longitude range for raindrops
height_range = [100, 500]  # Initial height range for raindrops

# Generate raindrops data
raindrops_data = generate_raindrops(num_drops, lat_range, lon_range, height_range)

# Define a ScatterplotLayer for raindrops
raindrop_layer = pdk.Layer(
    "ScatterplotLayer",
    raindrops_data,
    get_position="[lon, lat, height]",
    get_fill_color="[0, 0, 255, 150]",  # Blue color for raindrops
    get_radius="size",
    pickable=True,
    opacity=0.8,
)

# Define the initial view state
view_state = pdk.ViewState(latitude=37.75, longitude=-122.425, zoom=12, pitch=50)

# Create the pydeck Deck
deck = pdk.Deck(
    layers=[raindrop_layer],
    initial_view_state=view_state,
    tooltip={"text": "Raindrop at {lat}, {lon}"},
)

# Save the deck to HTML
deck.to_html("rain_effect.html")

# Generate the JavaScript code to animate the raindrops
animation_js = """
const animateRaindrops = () => {
    const raindrops = deckgl.layerManager.layers[0].props.data;
    raindrops.forEach(drop => {
        drop.height -= 10;
        if (drop.height < 0) {
            drop.height = Math.random() * 400 + 100;
        }
    });
    deckgl.layerManager.updateLayers();
    requestAnimationFrame(animateRaindrops);
};

document.addEventListener('DOMContentLoaded', () => {
    animateRaindrops();
});
"""

# Insert the JavaScript into the HTML file
with open("rain_effect.html", "r") as file:
    html_content = file.read()

html_content = html_content.replace(
    "</body>", f"<script>{animation_js}</script></body>"
)

with open("rain_effect.html", "w") as file:
    file.write(html_content)